In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import shutil
%matplotlib inline

In [41]:
#Define some constants
DATA_PATH ="C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//downloaded//"
TRAIN_PATH = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data//train//"
TEST_PATH = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data//test//"
VALIDATION_PATH = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data//validation//"
BEST_MODEL_PATH = "best_model.h5"
INPUT_SHAPE = (60,60,3)
BATCH_SIZE = 32
CLASS_MODE = 'categorical'
TRAINING_SIZE = 0.9 #train CNN with 90% of images
TESTING_SIZE = 0.05 #test CNN with 5% of images
VALIDATION_SIZE = 0.05 #validate CNN with 5% of images

# Data Preprocessing

In [17]:
print(len(os.listdir(DATA_PATH+"/paper")))
print(len(os.listdir(DATA_PATH+"/rock")))
print(len(os.listdir(DATA_PATH+"/scissors")))

def moveFiles(files,src,dest):
    for file in files:
        full_file_name = os.path.join(src, file)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name,dest)


def separateData():
    for move in ['paper','rock','scissors']:
        files = os.listdir(DATA_PATH+move)
        train_index = math.ceil(len(files)*0.9)
        test_index = train_index+ math.ceil(len(files)*0.05)
        validation_index = test_index+ math.floor(len(files)*0.05)
    
        moveFiles(files[:train_index],DATA_PATH+move,TRAIN_PATH+move)
        moveFiles(files[train_index:test_index],DATA_PATH+move,TEST_PATH+move)
        moveFiles(files[test_index:validation_index],DATA_PATH+move,VALIDATION_PATH+move)
    
    
    
    
    
    

712
726
750


In [55]:




data_generator = ImageDataGenerator(rescale=1. / 255,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

train_generator = data_generator.flow_from_directory(
    TRAIN_PATH,
    target_size=(60,60),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE
)


test_generator = data_generator.flow_from_directory(
    TEST_PATH,
    target_size=(60,60),
    batch_size=1,
    class_mode=CLASS_MODE
)




validation_generator = data_generator.flow_from_directory(
    VALIDATION_PATH,
    target_size=(60,60),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE
)




Found 1970 images belonging to 3 classes.
Found 111 images belonging to 3 classes.
Found 107 images belonging to 3 classes.


In [35]:
class CNN:
    def __init__(self, dataPath, inputShape):
        self.datapath = dataPath
        self.model = self.build_model(inputShape)
        self.model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
        
    
    def build_model(self, inputShape):
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=inputShape, activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.2))

        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.2))

        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(3, activation='softmax'))
        
        model.summary()
        return model

In [36]:
CNN = CNN(DATA_PATH, INPUT_SHAPE)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 58, 58, 64)        1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 26, 26, 128)       73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 128)      

In [37]:
checkpoint_callback = ModelCheckpoint(
    BEST_MODEL_PATH, 
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

reduce_callback  = ReduceLROnPlateau(
    monitor = 'val_accuracy',
    patience = 3,
    factor = 0.5,
    min_lr = 0.00001,
    verbose = 1
)

callbacks_list = [checkpoint_callback, reduce_callback]

In [45]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_val = validation_generator.n//validation_generator.batch_size
CNN.history = CNN.model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=30,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   verbose=1,
                   callbacks=callbacks_list)

Epoch 1/30
60/61 [============================>.] - ETA: 0s - loss: 1.0840 - accuracy: 0.3851
Epoch 00001: val_accuracy improved from -inf to 0.41667, saving model to best_model.h5
61/61 [==============================] - 9s 149ms/step - loss: 1.0841 - accuracy: 0.3854 - val_loss: 1.0794 - val_accuracy: 0.4167
Epoch 2/30
60/61 [============================>.] - ETA: 0s - loss: 1.0366 - accuracy: 0.4659
Epoch 00002: val_accuracy improved from 0.41667 to 0.50000, saving model to best_model.h5
61/61 [==============================] - 8s 138ms/step - loss: 1.0369 - accuracy: 0.4665 - val_loss: 1.0147 - val_accuracy: 0.5000
Epoch 3/30
60/61 [============================>.] - ETA: 0s - loss: 0.8107 - accuracy: 0.6369
Epoch 00003: val_accuracy improved from 0.50000 to 0.67708, saving model to best_model.h5
61/61 [==============================] - 9s 146ms/step - loss: 0.8090 - accuracy: 0.6388 - val_loss: 0.6528 - val_accuracy: 0.6771
Epoch 4/30
60/61 [============================>.] - ETA: 0

Epoch 28/30
60/61 [============================>.] - ETA: 0s - loss: 0.0385 - accuracy: 0.9873
Epoch 00028: val_accuracy did not improve from 1.00000
61/61 [==============================] - 9s 148ms/step - loss: 0.0380 - accuracy: 0.9875 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 29/30
60/61 [============================>.] - ETA: 0s - loss: 0.0238 - accuracy: 0.9932
Epoch 00029: val_accuracy did not improve from 1.00000
61/61 [==============================] - 9s 149ms/step - loss: 0.0234 - accuracy: 0.9933 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 30/30
60/61 [============================>.] - ETA: 0s - loss: 0.0386 - accuracy: 0.9880
Epoch 00030: val_accuracy did not improve from 1.00000

Epoch 00030: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
61/61 [==============================] - 9s 150ms/step - loss: 0.0381 - accuracy: 0.9882 - val_loss: 0.0019 - val_accuracy: 1.0000


# Testing out Model

In [46]:
CNN.model.load_weights(BEST_MODEL_PATH)

In [47]:
step_size_test = test_generator.n//test_generator.batch_size
testing_model = CNN.model.evaluate_generator(
    test_generator,
    step_size_test,
    verbose=1
)

3/3 [==============================] - 0s 118ms/step - loss: 0.0265 - accuracy: 0.9896


In [56]:
classes = CNN.model.predict_generator(test_generator, test_generator.n)

In [57]:
print(classes)

[[8.52282450e-04 9.92833257e-01 6.31444622e-03]
 [3.38490150e-04 1.04784488e-06 9.99660492e-01]
 [2.46258657e-02 9.74705279e-01 6.68860681e-04]
 [2.97104862e-06 9.99468267e-01 5.28828532e-04]
 [3.43536056e-04 9.99272406e-01 3.84104060e-04]
 [9.98943269e-01 3.49398033e-04 7.07369763e-04]
 [8.06592579e-04 1.56973942e-06 9.99191821e-01]
 [9.99999881e-01 1.34374188e-14 6.56784920e-08]
 [2.27364013e-03 2.15967031e-08 9.97726381e-01]
 [1.36999530e-03 9.95647371e-01 2.98259384e-03]
 [4.88483999e-03 8.80944754e-07 9.95114207e-01]
 [6.29544229e-05 9.99858737e-01 7.83045689e-05]
 [1.14855054e-03 1.31704310e-05 9.98838246e-01]
 [1.75255560e-03 7.28056375e-06 9.98240113e-01]
 [3.13880046e-05 9.99814570e-01 1.54075169e-04]
 [1.55645991e-02 6.76397467e-04 9.83758926e-01]
 [4.30244021e-04 9.99487281e-01 8.24578965e-05]
 [3.12988907e-02 9.00442600e-01 6.82585016e-02]
 [9.32812877e-03 6.32764946e-04 9.90039051e-01]
 [9.92160499e-01 5.42197376e-03 2.41755298e-03]
 [5.78590715e-03 9.77761865e-01 1.645219

In [54]:
len(classes)

111

In [58]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [59]:
len(test_generator.classes)

111

In [71]:
from keras.preprocessing import image
from PIL import Image

img = Image.open(TEST_PATH+'scissors//WYOb0YuOVCMs7ebJ.png')#, target_size(60,60))
img = img.resize((60,60)) 
img = image.img_to_array(img)
img = img.reshape((1,)+img.shape)

img_class = CNN.model.predict_classes(img)

In [72]:
print(img_class)

[2]
